In [8]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import io
from os.path import expanduser
HOME = expanduser("~")
import os, sys
import SimpleITK as sitk
from PIL import Image, ImageOps
from tqdm import tqdm
#%matplotlib inline
REPO_DIR = '/home/eddyod/programming/pipeline_utility'
sys.path.append(REPO_DIR)
from utilities.sqlcontroller import SqlController
from utilities.file_location import FileLocationManager
stack = 'DK39'
sqlController = SqlController()
fileLocationManager = FileLocationManager(stack)
image_name_list = sqlController.get_image_list(stack, 'destination')
files = [i for i in image_name_list if '_C1' in i]
#files = image_name_list[0:50]

Connecting dklab@192.168.1.12:3306


In [9]:
DIR = os.path.join(HOME, 'programming', 'dk39')
#DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK39'
ORIENTED = os.path.join(DIR, 'preps', 'oriented')
THUMBNAIL = os.path.join(DIR, 'preps', 'thumbnail')
TIF = os.path.join(DIR, 'tif')
INPUT = THUMBNAIL
#INPUTS = INPUTS[80:120]
COLOR = 2**16
files = [i for i in image_name_list if '_C1' in i]
files = os.listdir(INPUT)
#files = files[0:50]

In [12]:
def everything(img, rotation, filename):
    img = get_last_2d(img)
    img = np.rot90(img, rotation)
    #if 'DK39_ID_0358_slide030_S4_C1.tif' not in filename:
    #    img = crop_rows(img, 29)
    #else:
    #    print('Not cropping', filename)
    #maxi = np.amax(img) 
    #img = linnorm(img, maxi)
    return img.astype('uint16') 

def get_last_2d(data):
    if data.ndim <= 2:
        return data    
    m,n = data.shape[-2:]
    return data.flat[:m*n].reshape(m,n)

def crop_rows(img,cropy):
    y,x = img.shape
    starty = y - cropy
    return img[0:starty,:]

def lognorm(img, limit):
    lxf = np.log(img + 0.005)
    lxf = np.where(lxf < 0, 0, lxf)
    xmin = min(lxf.flatten())
    xmax = max(lxf.flatten())
    return -lxf * limit / (xmax - xmin) + xmax * limit / (xmax - xmin)  # log of data and stretch 0 to max


def linnorm(img, limit):
    flat = img.flatten()
    hist, bins = np.histogram(flat, limit + 1)
    cdf = hist.cumsum()  # cumulative distribution function
    cdf = limit * cdf / cdf[-1]  # normalize
    # use linear interpolation of cdf to find new pixel values
    img_norm = np.interp(flat, bins[:-1], cdf)
    img_norm = np.reshape(img_norm, img.shape)
    #img_norm = limit - img_norm
    return img_norm.astype('uint16')

def get_max_size(INPUT):
    widths = []
    heights = []
    files = os.listdir(INPUT)
    for file in files:
        img = io.imread(os.path.join(INPUT, file))
        heights.append(img.shape[0])
        widths.append(img.shape[1])

    max_width = max(widths)
    max_height = max(heights)

    return max_width, max_height


def get_max_intensity(INPUT):
    intensities = set()
    files = os.listdir(INPUT)
    for file in files:
        img = io.imread(os.path.join(INPUT, file))
        intensities.add(np.amax(img))
        
    return max(intensities),min()


In [13]:
# orient images
dels = os.listdir(ORIENTED)
#files = os.listdir(INPUT)
for d in dels:
    os.unlink(os.path.join(ORIENTED, d))
#for i in (files):
for i in tqdm(files):
    infile = os.path.join(INPUT, i)
    outfile = os.path.join(ORIENTED, i)
    img = io.imread(infile)
    img = everything(img, 3, i)
    io.imsave(outfile, img, check_contrast=False)
    img = None

100%|██████████| 460/460 [00:08<00:00, 51.15it/s]
